In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import datasets
import torch

In [13]:
!pip install datasets -q
!pip install -U bitsandbytes -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca")['train']
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 18612
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
from transformers import BitsAndBytesConfig

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

model_id = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             #quantization_config = bnb_config
                                            )

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [17]:
from transformers import TextStreamer

streamer = TextStreamer(tokenizer=tokenizer,
                        skip_special_tokens=True)


prompt ="""
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: Create a function to calculate the sum of a sequence of integers.
### Input: [1, 2, 3, 4, 5]
### Output:

"""

ids = tokenizer(prompt, return_tensors="pt").to(model.device)
out = model.generate(**ids,
                     streamer=streamer,
                     max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: Create a function to calculate the sum of a sequence of integers.
### Input: [1, 2, 3, 4, 5]
### Output:

To create a function that calculates the sum of a sequence of integers, you can follow these steps:

1. **Define the function**: Start by defining a function that takes an array of integers as its parameter.
2. **Initialize the sum**: Initialize a variable to hold the sum of the integers.
3. **Iterate through the array**: Use a loop to iterate through each element in the array.
4. **Add the element to the sum**: For each element, add it to the sum


## fine tune the gemma 2 2B model

In [18]:
!pip install peft -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(r=8,
                         task_type='CAUSAL_LM',
                         target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
)

# lora_model = get_peft_model(model, lora_config)

In [25]:
code_dataset = dataset.rename_column("prompt", "text")

In [28]:
code_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 18612
})

In [30]:
!pip install trl -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

args = TrainingArguments(output_dir="finetune-Qwen2-0.5B-codeDataset",
                per_device_train_batch_size=2,
                gradient_accumulation_steps=2,
                logging_steps=300,
                fp16=True,
                optim='paged_adamw_8bit',
                max_steps= 500
                )


trainer = SFTTrainer(
    model=model,
    train_dataset=code_dataset,
    tokenizer=tokenizer,
    args=args,
    dataset_text_field="text",
    max_seq_length=300,
    peft_config=lora_config
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [33]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
300,0.961700


TrainOutput(global_step=500, training_loss=0.9123429565429687, metrics={'train_runtime': 256.857, 'train_samples_per_second': 7.786, 'train_steps_per_second': 1.947, 'total_flos': 787309993147392.0, 'train_loss': 0.9123429565429687, 'epoch': 0.1074575542660649})

In [35]:
from huggingface_hub import notebook_login
notebook_login()

In [36]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

events.out.tfevents.1733494685.accf86df4ce3.23.0:   0%|          | 0.00/6.07k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlyGreo/finetune-Qwen2-0.5B-codeDataset/commit/ac57bb99aca63f8f3f5e36ebde1bb668ece244b1', commit_message='End of training', commit_description='', oid='ac57bb99aca63f8f3f5e36ebde1bb668ece244b1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AlyGreo/finetune-Qwen2-0.5B-codeDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='AlyGreo/finetune-Qwen2-0.5B-codeDataset'), pr_revision=None, pr_num=None)

## Test The model

In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tok = AutoTokenizer.from_pretrained("finetune-Qwen2-0.5B-codeDataset")
finetuned_model = AutoModelForCausalLM.from_pretrained("finetune-Qwen2-0.5B-codeDataset",
                                    device_map="auto",
                                    torch_dtype=torch.bfloat16)

In [54]:
def prompt_model(prompt, max_tokens=50):
    ids = tok(prompt, return_tensors="pt").to(finetuned_model.device)
    output = finetuned_model.generate(**ids, max_new_tokens=max_tokens)

    return tok.decode(output[0], skip_special_tokens=True)

In [55]:
prompt = """

### instruction : write a function that takes a list of values and return there sum

###input: [1,4,1,7,10]

###output: """

print(prompt_model(prompt, max_tokens=50))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




### instruction : write a function that takes a list of values and return there sum

###input: [1,4,1,7,10]

###output: 18

def sumValues(values):
    total = 0
    for value in values:
        total += value
    return total

print(sumValues([1,4,1,7,10])) # 18
print


In [56]:
prompt = """

### instruction : write a function in python to load a csv file in pandas and display shape dtypes and info

###input: "data.csv"

###output: """

print(prompt_model(prompt, max_tokens=50))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




### instruction : write a function in python to load a csv file in pandas and display shape dtypes and info

###input: "data.csv"

###output:  import pandas as pd

# open the csv file
data = pd.read_csv('data.csv')

# get the shape and dtype
print(data.shape)
print(data.dtypes)

# get the info
print(data.info) # get the info of


In [59]:
prompt = """

### instruction : write a function in python to merge 2 arrays in numpy and return the shape of the new matrix

###input: 

###output: """

print(prompt_model(prompt, max_tokens=35))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




### instruction : write a function in python to merge 2 arrays in numpy and return the shape of the new matrix

###input: 

###output: 2D numpy array
def merge_arrays(arr1, arr2):
    merged_array = np.concatenate([arr1, arr2], axis=1)
    return merged_array
